# Convert to Zarr files

In [1]:
import dask.bag as db
import dask

In [2]:
import xarray as xr
import fsspec

In [3]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)

In [4]:
from dask_cloudprovider.aws import FargateCluster
cluster = FargateCluster(image="ghcr.io/vietnguyengit/nc2zarr:main", scheduler_timeout="3600 minutes", task_role_arn="arn:aws:iam::615645230945:role/ManualDaskZarrCreation",
                         scheduler_cpu=4096, scheduler_mem=30720)

/home/ec2-user/anaconda3/envs/mxnet_latest_p37/lib/python3.7/contextlib.py:119: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


In [5]:
%%time
from dask.distributed import Client
client = Client(cluster)
display(client)

/home/ec2-user/anaconda3/envs/mxnet_latest_p37/lib/python3.7/site-packages/distributed/client.py:1100: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| blosc       | 1.10.6    | None      | None    |
| cloudpickle | 2.0.0     | 2.1.0     | None    |
| dask        | 2021.09.0 | 2022.02.0 | None    |
| distributed | 2021.09.0 | 2022.02.0 | None    |
| lz4         | 4.0.1     | None      | None    |
| msgpack     | 1.0.0     | 1.0.4     | None    |
| toolz       | 0.11.1    | 0.11.2    | None    |
+-------------+-----------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Connection method: Cluster object,Cluster type: dask_cloudprovider.FargateCluster
Dashboard: http://3.25.169.36:8787/status,
Dashboard: http://3.25.169.36:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.31.10.243:8786,Workers: 0
Dashboard: http://172.31.10.243:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


CPU times: user 32 ms, sys: 7.6 ms, total: 39.6 ms
Wall time: 279 ms


In [51]:
def open_zarr(r):
    m = fsspec.get_mapper(r)
    print(m)
    ds = xr.open_zarr(m)
    return ds

In [7]:
cluster.adapt(minimum=1,maximum=20)

distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=20


In [49]:
references = [path for path in (fsspec.filesystem('file').glob(f'/home/ec2-user/SageMaker/nc2zarr/outputs/argo/refs/1901126/*.json'))]

In [46]:
references[0]

'/home/ec2-user/SageMaker/nc2zarr/outputs/argo/refs/1901126/D1901126_001.json'

In [52]:
%%time
futures = []
for ref in references:
    futures.append(client.submit(open_zarr, ref, retries=10))

zarrs = [future.result() for future in futures]

GroupNotFoundError: group not found at path "group not found at path ''"

distributed.deploy.adaptive - INFO - Retiring workers [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]


In [11]:
# # zarrs = [z.drop_vars([], errors='ignore') for z in zarrs]
# ds = xr.concat(zarrs,dim='LATITUDE',coords='minimal',compat='override',combine_attrs='override', fill_value='')

# chunked = ds.chunk(chunks=64)

# for var in chunked.data_vars:
#     chunked[var].encoding = {}

# store= s3fs.S3Map(root=f's3://imos-data-pixeldrill/zarrs/viet-test/argo/from-dask/', s3=s3, check=False)

# delayed = chunked.to_zarr(store, mode='w', compute=False)
# delayed.compute()

distributed.deploy.adaptive - INFO - Retiring workers [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
